In [1]:
import re
import requests
from html.parser import HTMLParser

In [2]:
url_kabuyoho = 'https://kabuyoho.ifis.co.jp/index.php'
params = {'action': 'tp1',
          'sa': 'report_top',
          'bcode': '8035'}

In [3]:
response = requests.get(url_kabuyoho, params=params)
#print(response.text)

In [4]:
class ParserKabuYohoNews(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.pattern = re.compile(r'index\.php\?(action=tp1\&sa=consNewsDetail\&nid=.+)')
        self.result = list()
        self.element = None
        self.news = False
        self.date = False
        self.link = False

    def handle_starttag(self, tag, attrs):
        dict_attrs = dict(attrs)

        if tag == 'table':
            if 'class' in dict_attrs:
                if dict_attrs['class'] == 'tb_new_news':
                    self.news = True
        elif tag == 'span' and self.news is True:
            self.date = True
        elif tag == 'a' and self.news is True:
            if 'href' in dict_attrs:
                m = self.pattern.match(dict_attrs['href'])
                if m:
                    self.url = '%s?%s' % (url_kabuyoho, m.group(1))
                    self.link = True
        elif tag == 'tr' and self.news is True:
            self.element = list()

    def handle_data(self, data):
        data0 = data.strip()
        if self.news and self.date:
            self.element.append(data0)
            self.date = False

        if self.news and self.link:
            self.element.append(data0)
            self.element.append(self.url)
            self.link = False
            self.url = ''

    def handle_endtag(self, tag):
        if tag == 'table' and self.news is True:
            self.news = False
            return

        if tag == 'tr' and self.news is True:
            if len(self.element) > 0:
                self.result.append(self.element)
            return

In [5]:
parser = ParserKabuYohoNews()
parser.feed(response.text)
parser.close()

for line in parser.result:
    print(line)

['2024/05/20', '25年3月期経常予想。対前週0.7%上昇。', 'https://kabuyoho.ifis.co.jp/index.php?action=tp1&sa=consNewsDetail&nid=8035_20240520_cons_20240520_224532_62']
['2024/05/20', '米系大手証券、レーティング強気継続。目標株価引き上げ、48,000円。', 'https://kabuyoho.ifis.co.jp/index.php?action=tp1&sa=consNewsDetail&nid=8035_20240519_rep_20240520_110505_3']
['2024/05/16', '日系大手証券、レーティング強気継続。目標株価引き上げ、42,000円。', 'https://kabuyoho.ifis.co.jp/index.php?action=tp1&sa=consNewsDetail&nid=8035_20240516_rep_20240516_180007_13']
